In [ ]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GUTENBERG_CONFIG, GUTENBERG_DPO_CONFIG, SMOL_LM_135M

In [16]:
cfg = GUTENBERG_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_samples = 800
cfg.eval_steps = 5
cfg.eval_data_mode = "fixed"
wrapper = TrainerWrapper(cfg)

In [18]:
wrapper.init_model()

2025-02-22 21:48:28.264 | INFO     | trl_wrapper.trainer_wrapper:init_model:295 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: flash_attention_2


In [20]:
wrapper.init_data_module()

2025-02-22 21:48:30.077 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:316 - Using dataset path: dataset_files/gutenberg_conversations.parquet
2025-02-22 21:48:30.078 | INFO     | model.utils:__init__:118 - Cache dir: ../dataset_caches/conversation_data_module
2025-02-22 21:48:30.078 | INFO     | model.utils:setup:159 - Loading dataset for stage fit
2025-02-22 21:48:30.079 | INFO     | model.utils:setup:162 - Processing dataset for stage fit, workers: 1, cache dir ../dataset_caches/conversation_data_module, using cache: False
2025-02-22 21:48:30.079 | INFO     | model.utils:init_dataset:134 - Loading local dataset from ../dataset_files/gutenberg_conversations.parquet
2025-02-22 21:48:30.085 | INFO     | model.utils:init_dataset:144 - Selecting 800 samples
2025-02-22 21:48:30.089 | INFO     | model.utils:init_dataset:154 - Loading custom chat template: ../chat_templates/llama3.jinja
2025-02-22 21:48:30.089 | INFO     | model.utils:setup:171 - Train dataset samples: 720 Val da

In [23]:
wrapper.init_trainer()

2025-02-22 21:48:51.617 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:361 - Saving output to: ./runs/02-22-21-48-882269-smollm2-135m-instruct--gutenberg-conv
2025-02-22 21:48:51.621 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:384 - Initializing trainer, run_name: 02-22-21-48-882269-smollm2-135m-instruct--gutenberg-conv, wandb project: gutenberg
/home/brianf/smolmodels/notebooks/../trl_wrapper/trainer_wrapper.py:444: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `CustomSFTTrainer.__init__`. Use `processing_class` instead.
  self.trainer = CustomSFTTrainer(
Applying chat template to train dataset (num_proc=4): 100%|██████████| 720/720 [00:00<00:00, 1735.37 examples/s]
Applying chat template to eval dataset (num_proc=4): 100%|██████████| 80/80 [00:00<00:00, 482.62 examples/s]


In [22]:
wrapper.trainer.train_dataset

Dataset({
    features: ['input_ids'],
    num_rows: 720
})

In [33]:
wrapper.trainer.evaluate()

2025-02-22 22:02:05.686 | INFO     | trl_wrapper.sft_trainer:evaluation_loop:90 - Generating samples...
2025-02-22 22:02:05.687 | INFO     | trl_wrapper.sft_trainer:generate_from_model:56 - Generating policy samples, max length: 1024...
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
2025-02-22 22:02:05.926 | INFO     | trl_wrapper.sft_trainer:evaluation_loop:92 - Generated samples.
2025-02-22 22:02:05.972 | INFO     | synthetic_data.utils:log_to_file:327 - Saved eval samples.


┌────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┐
│ prompt                                             │ ref                                                │ policy                                             │
├────────────────────────────────────────────────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────┤
│ \n<|start_header_id|>system<|end_header_id|>Cuttin │ \n<|start_header_id|>system<|end_header_id|>Cuttin │ \n<|start_header_id|>system<|end_header_id|>Cuttin │
│ g Knowledge Date: December 2023Today Date: 22 Feb  │ g Knowledge Date: December 2023Today Date: 22 Feb  │ g Knowledge Date: December 2023Today Date: 22 Feb  │
│ 2025<|eot_id|><|start_header_id|>user<|end_header_ │ 2025<|eot_id|><|start_header_id|>user<|end_header_ │ 2025<|eot_id|><|start_header_id|>user<|end_header_ │
│ id|>Write a chapter set in Victo

{'eval_loss': 3.037513256072998,
 'eval_model_preparation_time': 0.0029,
 'eval_runtime': 2.655,
 'eval_samples_per_second': 30.132,
 'eval_steps_per_second': 15.066}

In [ ]:
wrapper.train()